# LGP defect detection in AI Core -  Part 1

The figure below summarizes the different steps one needs to go through to train and deploy a ML model in AI Core.

<img src="../../../../resources/AICoreMLOps.png" width="900"> 

In this notebook we assume that step 0.1 and 0.2 have been already done. We will guide you through:

* **Connecting a GitHub repository to an AI Core instance** using the AI API SDK (step 0.4 in the figure)
* **Connecting a Docker registry to an AI Core instance** using the AI API SDK (step 0.5 in the figure)
* **Uploading data on S3 and create a resource group** (step 1 in the figure)

<span style="color:red">For your reference, a complete example covering all the steps of the one-time AI Core configuration is available in </span>[this notebook](../notebooks/defect_detection_aicore_notebook_one_time_config.ipynb).


### Before getting started: color conventions

The comments within the notebook will guide you to the required steps. Pay attention to the color conventions:

* <span style="color:magenta"> **Magenta text**  </span> indicates that you have to open certain json files and modify them according to your own set up, for instance you can be asked to enter credentials for a certain system, change names for the variables etc.  
* <span style="color:blue"> **Blue text**  </span> indicates that you have to execute commands on a terminal. 
* <span style="color:green"> **Green text** </span> indicates that you are asked to modify something in the following notebook cells. 


## Create an AI API client instance

In [ ]:
from ai_api_client_sdk import ai_api_v2_client
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
import json

In [ ]:
from ai_api_client_sdk import ai_api_v2_client

First of all, we need to create an AI API client instance, which will allow us to interact with our SAP AI Core tenant. <span style="color:magenta">Before executing the code, edit </span>**[aic_service_key.json](./files/aic_service_key.json)**<span style="color:magenta"> and enter the right credentials of your AI Core tenant</span>.

In [ ]:
# Your service key JSON file relative to your aicore instance
aic_service_key_path = './files/aic_service_key.json'

# Loads the service key file
with open(aic_service_key_path) as ask:
    aic_service_key = json.load(ask)

# Creating an AI API client instance
ai_api_client = AIAPIV2Client(
    base_url = aic_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aic_service_key["url"] + "/oauth/token",
    client_id = aic_service_key['clientid'],
    client_secret = aic_service_key['clientsecret']
)

##  Connect your GitHub repository to SAP AI Core

Now, let's use the AI API client to connect your GitHub repository to the AI Core instance. <span style="color:magenta">Before executing the code, edit </span> **[git_setup.json](./files/git_setup.json)** <span style="color:magenta">with your GitHub credentials (for the moment we focus on the first part of the json)</span>:
* <span style="color:magenta">name:</span> complete with the name that will identify the GitHub connection within AI Core
* <span style="color:magenta">URL:</span> your GitHub repository url
* <span style="color:magenta">username:</span> your GitHub username
* <span style="color:magenta">password:</span> your GitHub access token
</span>.

In [ ]:
git_setup_file_path = "./files/git_setup.json"

In [ ]:
# Loads your git_setup.json
with open(git_setup_file_path) as gs:
    setup_json = json.load(gs)

# Connects your repo
repo_json = setup_json["repo"]
response = ai_api_client.rest_client.post(
    path="/admin/repositories",
    body={
        "name": repo_json["name"],
        "url": repo_json["url"],
        "username": repo_json["username"],
        "password": repo_json["password"]
    }
)
print(response)

Check out if the repository has been registered correctly:

In [ ]:
ai_api_client.rest_client.get(
    path="/admin/repositories"
)

##  Connect your Docker Registry to SAP AI Core

Next, let's use the AI API client to connect your Docker registry to the AI Core instance. <span style="color:magenta">Before executing the code, edit </span> **[docker_secret.json](./files/docker_secret.json)** <span style="color:magenta">with your docker credentials</span>:
* <span style="color:magenta">username:</span> your docker username
* <span style="color:magenta">password:</span> your docker access token
</span>.

In [ ]:
docker_secret_file_path = './files/docker_secret.json'

# Loads the json file
with open(docker_secret_file_path) as dsf:
    docker_secret = json.load(dsf)

response = ai_api_client.rest_client.post(
    path="/admin/dockerRegistrySecrets",
    body={
        "name": docker_secret["name"],
        "data": docker_secret["data"]
    }
)
print(response)

## Upload data on S3 and connect them to a Resource Group

### Create a new Resource Group

Resource Groups represent a virtual collection of related resources within the scope of one SAP AI Core tenant. When you create a new AI Core instance in BTP, a "default" resource group is created automatically. Optionally, you can create more resource groups to keep segregated AI assets related to different use cases, development teams, customers, etc. <span style="color:red">Please, notice that if you are working with the free tier AI Core, you are not allowed to create further resource groups, so you are limited to use the default one and there is nothing to be done in this section. </span> For an example on how to create a resource group, you can check out the instructions for the standard AI Core instance [here](https://github.com/SAP-samples/btp-ai-sustainability-bootcamp/blob/opensap-standard/src/ai-models/defect-detection/exercises/defect-detection-part1.ipynb). 

### Upload the data on AWS S3 and register it to the Resource Group

In this exercise we will use AWS S3 Object Store as a cloud storage for our AI Core datasets and models. If you went through the **[prerequisites](../../../../prerequisites/prerequisites.md)**, you have already set up an S3 storage bucket and uploaded the image dataset.
Now you need to **register the Object Store to SAP AI Core resource group**. In order to do that we prepare a json file with the needed AWS S3 credentials and we use a proper AI API in order to register it. <span style="color:magenta"> Please enter your AWS S3 Object Store credentials in  </span>  **[s3_service_key.json](./files/s3_service_key.json)** before executing the following code. 


In [ ]:
# Loads your service key
s3_service_key_path = "./files/s3_service_key.json"

# Loads the service key file
with open(s3_service_key_path) as s3sk:
    s3_service_key = json.load(s3sk)

default_secret = {
    "name": "default",
    "type": "S3",
    "endpoint": s3_service_key["host"],
    "bucket": s3_service_key["bucket"],
    "pathPrefix": "image",
    "region": s3_service_key["region"],
    "data": {
        "AWS_ACCESS_KEY_ID": s3_service_key["access_key_id"],
        "AWS_SECRET_ACCESS_KEY": s3_service_key["secret_access_key"]
    }
}

# Call the api
ai_api_client.rest_client.post(
    path="/admin/objectStoreSecrets",
    body = default_secret, # defined above
    resource_group = "default"
)

Check out if the object store secret has been registered correctly with the following API call:

In [ ]:
ai_api_client.rest_client.get(
    path="/admin/objectStoreSecrets",
    resource_group = "default"
)

### Check out [defect-detection-part2.ipynb](./defect-detection-part2.ipynb) to continue with the exercise.